<a href="https://colab.research.google.com/github/Sohila912/CNN_Data_Augmentation/blob/main/Data_aug1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import files

# # Upload the zip file
# uploaded = files.upload()


In [2]:
import zipfile
import os

# Unzipping the uploaded dataset
with zipfile.ZipFile("/content/cat_dog.zip", "r") as zip_ref:
    zip_ref.extractall("cat_dog1")

# Check if the dataset is extracted
os.listdir("cat_dog1")


['train_sub', 'testsub']

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [4]:
img = load_img("/content/catordog.jpg")
x = img_to_array(img) # Numpy array with shape (150, 150, 3)
x = x.reshape((1,) + x.shape) # Reshape to (1, 3, 150, 150)

In [5]:
#from google.colab import drive
#drive.mount('/content/drive')

In [5]:
# i =0
# for batch in datagen.flow(x, batch_size=1,
#                             save_to_dir=r"E:\Python\DL\Data_Augmentation\preview",
#                             save_prefix='cat',
#                             save_format='jpeg'):
#         i += 1
#         if i > 30:  # Generate 20 images only
#             break

# Create the directory inside Colab
save_dir = "/content/preview"
os.makedirs(save_dir, exist_ok=True)

# Use the new directory in your datagen
i = 0
for batch in datagen.flow(x, batch_size=1,
                            save_to_dir=save_dir,  # Updated path
                            save_prefix='cat',
                            save_format='jpeg'):
    i += 1
    if i > 30:  # Generate 30 images only
        break


In [6]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense


In [7]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [9]:
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        r"/content/cat_dog/train_sub",  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        r"/content/cat_dog/testsub",
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

Found 2488 images belonging to 2 classes.
Found 260 images belonging to 2 classes.


In [10]:
model.fit(
    train_generator,
    steps_per_epoch=2000 // batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=800 // batch_size
)

# Save the trained weights with the correct filename
model.save_weights('first_try.weights.h5')  # Filename now follows the required convention


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 20s 128ms/step - accuracy: 0.6713 - loss: 0.6736 - val_accuracy: 0.6308 - val_loss: 0.7377
Epoch 2/50
  3/125 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - accuracy: 0.6528 - loss: 0.6097

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.6929 - loss: 0.6158 - val_accuracy: 0.6308 - val_loss: 0.6238
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 32s 104ms/step - accuracy: 0.7080 - loss: 0.5891 - val_accuracy: 0.6577 - val_loss: 0.5862
Epoch 4/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.7316 - loss: 0.5666 - val_accuracy: 0.6346 - val_loss: 0.6135
Epoch 5/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 13s 104ms/step - accuracy: 0.7157 - loss: 0.5521 - val_accuracy: 0.6346 - val_loss: 0.6116
Epoch 6/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.7237 - loss: 0.5528 - val_accuracy: 0.6308 - val_loss: 0.6111
Epoch 7/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 13s 104ms/step - accuracy: 0.7348 - loss: 0.5416 - val_accuracy: 0.7154 - val_loss: 0.5446
Epoch 8/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.7423 - loss: 0.5208 - val_accuracy: 0.7000 - val_loss: 0.5541
Epoch 9/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 13s 104ms/step - accuracy: 0.7339 - loss: 0.5273 - val_accurac

In [ ]:
model.fit(
    train_generator,
    steps_per_epoch=2000 // batch_size,
    epochs=95,  # Total epochs
    initial_epoch=50,  # Start from epoch 50
    validation_data=validation_generator,
    validation_steps=800 // batch_size
)
# Save the trained weights with the correct filename
model.save_weights('first_try_updated.weights.h5')  # Filename now follows the required convention

Epoch 51/95
125/125 ━━━━━━━━━━━━━━━━━━━━ 14s 109ms/step - accuracy: 0.8695 - loss: 0.3578 - val_accuracy: 0.8115 - val_loss: 0.4728
Epoch 52/95
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - accuracy: 0.8717 - loss: 0.2909 - val_accuracy: 0.8192 - val_loss: 0.5248
Epoch 53/95
125/125 ━━━━━━━━━━━━━━━━━━━━ 16s 106ms/step - accuracy: 0.8672 - loss: 0.3121 - val_accuracy: 0.8192 - val_loss: 0.4388
Epoch 54/95
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.8635 - loss: 0.3225 - val_accuracy: 0.8000 - val_loss: 0.4534
Epoch 55/95
125/125 ━━━━━━━━━━━━━━━━━━━━ 17s 106ms/step - accuracy: 0.8556 - loss: 0.3190 - val_accuracy: 0.8077 - val_loss: 0.5025
Epoch 56/95
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.8651 - loss: 0.3599 - val_accuracy: 0.8346 - val_loss: 0.4705
Epoch 57/95
125/125 ━━━━━━━━━━━━━━━━━━━━ 17s 105ms/step - accuracy: 0.8808 - loss: 0.3325 - val_accuracy: 0.7923 - val_loss: 0.4796
Epoch 58/95
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - accuracy: 0.8361 - loss: 0